In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/train.csv')

In [ ]:
data_train.head()

In [ ]:
data_train=data_train.drop('Instance_ID', axis =1)

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train.dtypes

In [ ]:
# How many category in target
data_train['Action'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le  = LabelEncoder()
data_train['Action'] = le.fit_transform(data_train['Action'])

In [ ]:
data_train['Action'].value_counts()

In [ ]:
X = data_train.drop('Action', axis =1)
Y= data_train['Action']

In [ ]:
# Apply standard scaller

from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()

X = ss.fit_transform(X)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    # Hidden Layer   (units = no of input(47) + no of output(3))
    tf.keras.layers.Dense(units= 49, activation = 'relu',input_shape= (X.shape[1],)),
    # output layers
    tf.keras.layers.Dense(units =3, activation = 'softmax')
])

In [ ]:
# Here we have muliclass in target 
# and also it is unbalanced then we use SMOTE inbuiled class to balance data

from imblearn.over_sampling import SMOTE

X1,Y1 = SMOTE().fit_resample(X,Y)

In [ ]:
Y1.value_counts()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# create a procedure for EARLY STOPING
# calling inbuilt class:   EarlyStopping

from tensorflow.keras.callbacks import EarlyStopping

#create object  
cb = EarlyStopping(monitor = 'val_loss',   # Mention parameter to monitor .. it may me loss or score
                   min_delta = 0.0001,     # diff btw new and previous loss: bydefault we take 0.0001
                   patience = 20,          
                   verbose =1,
                   mode = 'auto',
                   baseline = None,
                   restore_best_weights =False)

In [ ]:
# train the model: use inbuilt method fit() of Sequential class

train_model = model.fit(X1, Y1,epochs =5000,validation_split=0.2, callbacks = cb, batch_size = 64)
# validation_split = 0.2 :: It is used when ever you do not have output data in test file. In this case
# we are using 20% of train data to test the model

In [ ]:
# Here we see that out model is Underfit ....No we will increse the no of Hidden layer
print("Traning Loss and Accuracy: ", model.evaluate(X1, Y1))


In [ ]:
# Visulisation
plt.plot(train_model.history['loss'],label = 'Traning Loss')
plt.plot(train_model.history['val_loss'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('sparse_categorical_crossentropy')
plt.legend()
plt.show()

In [ ]:
# Visulisation
plt.plot(train_model.history['accuracy'],label = 'Traning Loss')
plt.plot(train_model.history['val_accuracy'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# To load test file 
data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/test.csv')

In [ ]:
data_test.head()

In [ ]:
data_test.drop('Instance_ID',axis =1,inplace = True)

In [ ]:

# Make a copy
X_test = data_test


In [ ]:
# Apply scaling
X_test = ss.transform(X_test)

In [ ]:
# prediction

Y_pred = model.predict(X_test).round(2)
Y_pred

In [ ]:
Y_pred = [np.argmax(i) for i in Y_pred]

Y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
Y_pred1 = []
for i in Y_pred:
  if i == 0:
    Y_pred1.append('Action_1')
  elif i ==1:
    Y_pred1.append('Action_2')
  else:
    Y_pred1.append('Other_Action')

In [ ]:
Y_pred1

In [ ]:
data_test['Action'] = Y_pred1

In [ ]:
data_test

In [ ]:
data_test.to_csv('testml', index =False)  # to save Data in csv